# Overview

Both SPRITE and NPC have a theta calculation. Are they related? Which is faster?

In [1]:
import math
import random

in NPC
```python
   def get_theta(self, xpos: float, ypos: float):
        """
        Calculate the theta, postheta, and side for the NPC
        :param xpos:
        :param ypos:
        :return:
        """
        theta = math.atan2(-ypos, xpos) % (2 * math.pi)
        theta = math.degrees(theta)

        self.postheta = theta
        theta -= self.face

        if theta < 0:
            theta += 360
        elif theta > 360:
            theta -= 360

        self.theta = theta
        self.set_side(self.directional_sprite.get_side_for_theta(theta))

        return theta
```

In [2]:
 def npc_theta(xpos: float, ypos: float):
    """
    Calculate the theta, postheta, and side for the NPC
    :param xpos:
    :param ypos:
    :return:
    """
    theta = math.atan2(-ypos, xpos) % (2 * math.pi)
    theta = math.degrees(theta)

    if theta < 0:
        theta += 360
    elif theta > 360:
        theta -= 360

    return theta

in SPRITE

```python
   def get_pos(self):
        """
        Determine position relative to the player.
        :return:
        """
        angle = SETTINGS.player_angle
        fov = SETTINGS.fov

        xpos = self.rect.centerx - SETTINGS.player_rect[0]
        ypos = SETTINGS.player_rect[1] - self.rect.centery

        dist = math.sqrt(xpos * xpos + ypos * ypos)
        if dist == 0:
            dist += 0.0001
        self.distance = dist

        thetaTemp = math.atan2(ypos, xpos)
        thetaTemp = math.degrees(thetaTemp)
        if thetaTemp < 0:
            thetaTemp += 360

        self.theta = thetaTemp

        yTmp = angle + (fov / 2) - thetaTemp
        if thetaTemp > 270 and angle < 90:
            yTmp = angle + (fov / 2) - thetaTemp + 360
        if angle > 270 and thetaTemp < 90:
            yTmp = angle + (fov / 2) - thetaTemp - 360

        xTmp = yTmp * SETTINGS.canvas_actual_width / fov

        sprite_height = int((self.rect.height / dist) * (100 / math.tan(math.radians(fov * 0.8))))
        if sprite_height > 2500:
            sprite_height = 2500

        sprite_width = int(self.rect.width / self.rect.height * sprite_height)

        if xTmp > (0 - sprite_width) and xTmp < (SETTINGS.canvas_actual_width + sprite_width):
            SETTINGS.zbuffer.append(self)

            if self.parent:
                self.parent.in_canvas = True
        else:
            if self.parent:
                self.parent.in_canvas = False

        self.new_size = pygame.transform.scale(self.texture, (sprite_width, sprite_height))
        self.new_rect = self.new_size.get_rect()
        self.new_rect.center = (xTmp, SETTINGS.canvas_target_height / 2)
        if self.parent:
            self.parent.hit_rect = self.new_rect
```

In [3]:
def sprite_theta(xpos, ypos):
    """
    Determine position relative to the player.
    :return:
    """
    dist = math.sqrt(xpos * xpos + ypos * ypos)
    if dist == 0:
        dist += 0.0001    

    thetaTemp = math.atan2(ypos, xpos)
    thetaTemp = math.degrees(thetaTemp)
    if thetaTemp < 0:
        thetaTemp += 360

    return thetaTemp


Helper function for generatingsome co-ordinates

In [4]:
def get_co_ords(size):
    x_s = [x for x in random.sample(list(range(size+100)), size)]
    y_s = [y for y in random.sample(list(range(size+100)), size)]
    
    return list(zip(x_s, y_s))

In [5]:
test_co_ords = get_co_ords(5)

Adding the result of NPC theta and sprite theta we get 360. As NPC is caluculating the inverse as to facing the camera.

In [6]:
for pos in test_co_ords:
    npc_theta_result = npc_theta(pos[0], pos[1])
    sprite_theta_result = sprite_theta(pos[0], pos[1])
    
    print("")
    print(pos)
    print("npc_theta_result", npc_theta_result)
    print("sprite_theta_result", sprite_theta_result)
    print(sprite_theta_result + npc_theta_result)


(103, 103)
npc_theta_result 315.0
sprite_theta_result 45.0
360.0

(104, 76)
npc_theta_result 323.84181456019166
sprite_theta_result 36.15818543980833
360.0

(66, 57)
npc_theta_result 319.18491612511843
sprite_theta_result 40.81508387488159
360.0

(50, 37)
npc_theta_result 323.49855887949366
sprite_theta_result 36.501441120506314
360.0

(4, 22)
npc_theta_result 280.304846468766
sprite_theta_result 79.69515353123397
360.0


In [7]:
%timeit npc_theta(pos[0], pos[1])

The slowest run took 10.60 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 670 ns per loop


In [8]:
%timeit sprite_theta(pos[0], pos[1])

The slowest run took 9.09 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 737 ns per loop


Some minor difference for a single co-ordinates, lets see what happens with many co-ordinates

In [9]:
def test_npc_theta(co_ords):
    return [npc_theta(pos[0], pos[1]) for pos in co_ords]            

In [10]:
def sprite_theta_theta(co_ords):
    return [sprite_theta(pos[0], pos[1]) for pos in co_ords]

In [11]:
test_co_ords = get_co_ords(30)

In [12]:
%timeit test_npc_theta(test_co_ords)

100000 loops, best of 3: 19.4 µs per loop


In [13]:
%timeit sprite_theta_theta(test_co_ords)

The slowest run took 4.46 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 22.5 µs per loop


# Numpy

Can we use numpy to speed things along?

In [14]:
import numpy as np

```
    theta = math.atan2(-ypos, xpos) % (2 * math.pi)
    theta = math.degrees(theta)

    if theta < 0:
        theta += 360
    elif theta > 360:
        theta -= 360
```

In [15]:
def get_np_co_ords(co_ords):
    x = np.array([pos[0] for pos in co_ords])
    y = np.array([pos[1] for pos in co_ords])
    return x, y

In [16]:
co_ords = get_co_ords(100)
entities_x, entities_y = get_np_co_ords(co_ords)

as per http://lagrange.univ-lyon1.fr/docs/numpy/1.11.0/reference/generated/numpy.arctan2.html#numpy.arctan2, y goes first

In [17]:
def np_theta(x, y):
    return 360 - np.arctan2(y ,x) * (180 / np.pi)

In [18]:
np_result = np_theta(entities_x, entities_y)
np_result

array([ 277.9071627 ,  333.43494882,  278.4146817 ,  289.15561245,
        304.50852299,  295.51387043,  339.78973078,  348.40782459,
        314.12532323,  318.43363036,  289.67409857,  289.68332755,
        319.56322975,  291.58979169,  352.67359334,  299.39605285,
        283.82205464,  304.93400897,  285.19524853,  344.40718906,
        288.43494882,  300.03328044,  291.26905984,  320.94151979,
        357.61405597,  306.57303098,  293.30884525,  301.32869287,
        322.52382044,  299.08192683,  284.8815297 ,  343.79622579,
        322.45024907,  350.34010692,  327.52880771,  287.19854122,
        313.74919919,  311.60028584,  335.15761089,  291.65043281,
        331.64503821,  352.52266969,  309.56746013,  353.76291992,
        319.17298448,  327.61932229,  313.50566641,  286.94922423,
        322.28402365,  329.03624347,  333.43494882,  289.7326895 ,
        359.36692323,  275.11731485,  273.69138599,  357.29512401,
        318.50353164,  271.90915243,  306.43548136,  344.90447

In [19]:
npc_result = test_npc_theta(co_ords)
npc_result

[277.90716270295843,
 333.434948822922,
 278.4146817003066,
 289.15561244543966,
 304.50852298766836,
 295.51387042753424,
 339.7897307848428,
 348.4078245897089,
 314.1253232288787,
 318.4336303624505,
 289.6740985722506,
 289.68332754644103,
 319.56322975423785,
 291.58979168771396,
 352.67359333983046,
 299.39605285494326,
 283.8220546357317,
 304.93400896676536,
 285.19524852681985,
 344.40718906073363,
 288.434948822922,
 300.0332804359951,
 291.26905983922853,
 320.9415197937528,
 357.61405596961123,
 306.5730309785193,
 293.3088452451268,
 301.3286928678042,
 322.5238204386386,
 299.0819268340175,
 284.88152970314053,
 343.79622578510924,
 322.4502490686562,
 350.34010692155766,
 327.5288077091515,
 287.19854122006586,
 313.7491991852346,
 311.6002858408767,
 335.1576108945448,
 291.65043281467024,
 331.64503821467594,
 352.5226696933392,
 309.56746013262295,
 353.7629199235415,
 319.17298447607953,
 327.61932229343074,
 313.5056664087334,
 286.94922423382144,
 322.2840236544168

So so some variation whats the damage?

In [27]:
for index, val in enumerate(npc_result):
    print(val, np_result[index], float(abs(val - np_result[index])))

277.90716270295843 277.907162703 0.0
333.434948822922 333.434948823 0.0
278.4146817003066 278.4146817 0.0
289.15561244543966 289.155612445 0.0
304.50852298766836 304.508522988 5.684341886080802e-14
295.51387042753424 295.513870428 0.0
339.7897307848428 339.789730785 0.0
348.4078245897089 348.40782459 0.0
314.1253232288787 314.125323229 0.0
318.4336303624505 318.433630362 0.0
289.6740985722506 289.674098572 0.0
289.68332754644103 289.683327546 0.0
319.56322975423785 319.563229754 0.0
291.58979168771396 291.589791688 0.0
352.67359333983046 352.67359334 0.0
299.39605285494326 299.396052855 0.0
283.8220546357317 283.822054636 5.684341886080802e-14
304.93400896676536 304.934008967 5.684341886080802e-14
285.19524852681985 285.195248527 5.684341886080802e-14
344.40718906073363 344.407189061 0.0
288.434948822922 288.434948823 0.0
300.0332804359951 300.033280436 5.684341886080802e-14
291.26905983922853 291.269059839 0.0
320.9415197937528 320.941519794 0.0
357.61405596961123 357.61405597 5.68434

In [29]:
5.684341886080802e-14 + 1

1.0000000000000568

So apart from a negliable degree of inaccuracy aside is there any speed benefit?

In [21]:
%timeit 360 - np.arctan2(entities_y ,entities_x) * (180 / np.pi)

The slowest run took 8.11 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.14 µs per loop


In [22]:
%timeit test_npc_theta(co_ords)

10000 loops, best of 3: 63.5 µs per loop


So if we increase the number of co-ordinates

In [23]:
co_ords = get_co_ords(1000)
entities_x, entities_y = get_np_co_ords(co_ords)

In [24]:
%timeit 360 - np.arctan2(entities_y ,entities_x) * (180 / np.pi)

10000 loops, best of 3: 29.7 µs per loop


In [25]:
%timeit test_npc_theta(co_ords)

1000 loops, best of 3: 669 µs per loop


the npc theta calc increased in a linear fashion, 10 times as longer for 10 times as many items.
numpy calculation did not

# Conclusion

Apart from some trig to iron out. It may be worthwhile to store npcs, sprite, walls - map positions in a vector fashion do distance and angle calculations using numpy.